* Check if the implicit velocity operator $A$ is symmetric positive-definite.
* Check if there is any advantage in formulating the operators such that $D = -G^T$ and $H f = -E^T \tilde{f}$.

In [ ]:
import numpy
from scipy.sparse import csc_matrix, csr_matrix, hstack, identity
from scipy.sparse.linalg import inv

import pyibm

from helper import *

In [ ]:
%matplotlib qt

In [ ]:
pyibm.__version__

In [ ]:
xlim, ylim = (-15.0, 15.0), (-15.0, 15.0)
xbox, ybox = (-0.75, 0.75), (-0.75, 0.75)
dx, dy = 0.05, 0.05  # grid spacings in the uniform region

config = {'x': {'start': xlim[0],
                'segments': [{'end': xbox[0],
                              'width': dx,
                              'stretching': 1.03,
                              'reverse': True},
                             {'end': xbox[1],
                              'width': dx,
                              'stretching': 1.0},
                             {'end': xlim[1],
                              'width': dx,
                              'stretching': 1.01}]},
          'y': {'start': ylim[0],
                'segments': [{'end': ybox[0],
                              'width': dy,
                              'stretching': 1.04,
                              'reverse': True},
                             {'end': ybox[1],
                              'width': dy,
                              'stretching': 1.0},
                             {'end': ylim[1],
                              'width': dy,
                              'stretching': 1.04}]}}

# Comment when not testing.
# config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
#               y=dict(start=-2.0, end=2.0, num_cells=20))
# dx, dy = 4.0 / 20, 4.0 / 20

grid = pyibm.GridBase(config=config)
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)

In [ ]:
print(gridc)
print(gridx)
print(gridy)

In [ ]:
delta_kernel = pyibm.delta_roma_et_al_1999
delta_kernel_size = 2

In [ ]:
GHat = pyibm.assemble_GHat(gridc, gridx, gridy)
DHat = pyibm.assemble_DHat(gridc, gridx, gridy)

print('GHat:')
print_matrix_info(GHat)
print('DHat:')
print_matrix_info(DHat)

In [ ]:
MHat = pyibm.assemble_MHat(gridx, gridy)
R = pyibm.assemble_R(gridx, gridy)
RInv = pyibm.diagonal_inv(R)
M = MHat @ RInv

In [ ]:
G = MHat @ GHat
D = DHat @ RInv

print('G:')
print_matrix_info(G)
print('D:')
print_matrix_info(D)

# Check if divergence is the opposite
# of the transpose of the gradient.
K = D + G.T
K = K.multiply(abs(K) > 1e-12) # remove extremely small values
assert K.nnz == 0

Indeed, we have $D = -G^T$ with their elements equal to $+1$ or $-1$.

In [ ]:
radius = 0.5
xc, yc = 0.0, 0.0
ds = dx
N = int(round(2 * numpy.pi * radius / ds))
epsilon = 0.0
theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1] + epsilon
x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
body = pyibm.Body(x, y, grid=gridc)
body

In [ ]:
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=delta_kernel,
                          kernel_size=delta_kernel_size)
print_matrix_info(Op)

alpha = dx * dy
EHat = alpha * Op

# Another way to compute EHat.
# Interpolation operator implemented in the decoupled IBPM of PetIBM.
EHat2 = Op @ R @ MHat
assert numpy.all((EHat - EHat2).data < 1e-12)

E = EHat @ RInv

beta = ds
HHat = beta / alpha * csr_matrix(EHat.T)
H = MHat @ HHat

# Spreading operator implemented in the decoupled IBPM of PetIBM.
HHat2 = csr_matrix(Op.T)

In [ ]:
ux = 2 * numpy.ones(gridx.size)
uy = numpy.ones(gridy.size)
u = numpy.concatenate((ux, uy))

U = EHat @ u
Ux, Uy = U[::body.ndim], U[1::body.ndim]

assert numpy.all(Ux - 2 < 1e-12)
assert numpy.all(Uy - 1 < 1e-12)

In [ ]:
Fx = 2 * numpy.ones(body.size)
Fy = numpy.ones(body.size)
F = numpy.empty(body.ndim * body.size)
F[::body.ndim], F[1::body.ndim] = Fx, Fy

f = HHat @ F
fx, fy = f[:gridx.size], f[gridx.size:]

F2 = EHat @ f
Fx2, Fy2 = F2[::body.ndim], F2[1::body.ndim]


print(numpy.sum(Fx) * beta, numpy.sum(Fy) * beta)
print(numpy.sum(fx) * alpha, numpy.sum(fy) * alpha)
print(numpy.sum(Fx2) * beta, numpy.sum(Fy2) * beta)

In [ ]:
plot_contourf(fx.reshape(gridx.shape), gridx, body=body,
              axis_lim=(-2.0, 2.0, -2.0, 2.0),
              show_grid=True);

In [ ]:
B = HHat @ EHat
print_matrix_info(B)
# plot_matrix(B);

In [ ]:
Re = 100.0
LHat = 1 / Re * pyibm.assemble_LHat(gridx, gridy)
print_matrix_info(LHat)

In [ ]:
L = MHat @ LHat @ RInv

dt = 0.01
I = identity(gridx.size + gridy.size)

alpha_implicit = 0.5
AHat = 1 / dt * I - alpha_implicit * LHat

A = MHat @ AHat @ RInv
is_symmetric(A)

In [ ]:
# Check if A is SPD.
# lambda_min ,lambda_max, cond = condition_number(A)
# lambda_min > 0

In [ ]:
BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=3, L=L, M=M)
print_matrix_info(BN)
# print('Condition number: ', condition_number(BN))

In [ ]:
Q = hstack([G, E.T])
print_matrix_info(Q)

In [ ]:
QTBNQ = Q.T @ BN @ Q
print_matrix_info(QTBNQ)
# print('Condition number: ', condition_number(QTBNQ))
# plot_matrix(QTBNQ, cmap='viridis')
is_symmetric(QTBNQ)

In [ ]:
GTBNG = G.T @ BN @ G
print_matrix_info(GTBNG)
# print('Condition number: ', condition_number(GTBNG))
# plot_matrix(GTBNG, cmap='viridis')
is_symmetric(GTBNG)

In [ ]:
DBNG = D @ BN @ G
print_matrix_info(DBNG)
is_symmetric(DBNG)

In [ ]:
DBNG = DHat @ BN @ GHat
print_matrix_info(DBNG)
is_symmetric(DBNG)

In [ ]:
K = DBNG - DBNG.T
K = K.multiply(abs(K) > 1e-3)
print(K.nnz)
# plot_matrix(K, cmap='viridis_r')

`DBNG` is not symmetric.
Could it be a bug in my code?
It may be because of not using ghost cells outside the external boundaries.

In [ ]:
# Condition number when using E and H.
for i in range(1, 3 + 1):
    print('Expansion order: ', i)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=i, L=L, M=M)
    EBNH = E @ BN @ H
    print_matrix_info(EBNH)
    print(condition_number(EBNH))

Condition number not really changing at we use a high-order expansion of the approximate inverse.

In [ ]:
# Condition number when using EHat and HHat.
for i in range(1, 3 + 1):
    print('Expansion order: ', i)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=i, L=LHat, M=MHat)
    EBNH = EHat @ BN @ HHat
    print_matrix_info(EBNH)
    print(condition_number(EBNH))

Again, no real change in the condition number.
So, what's the advantage of scaling `EHat` and `HHat`.

In [ ]:
# Condition number of EBNH implemented in PetIBM.
# How is it affected by the Taylor expansion order?
for i in range(1, 3 + 1):
    print('Expansion order: ', i)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=i, L=LHat, M=MHat)
    EBNH = EHat2 @ BN @ HHat2
    print_matrix_info(EBNH)
    print(condition_number(EBNH))
    # plot_matrix(EBNH_t, cmap='viridis');

The condition number of the operator `EBNH` is not significantly affected by the order of the Taylor expansion used to compute the inverse of the implicit velocity operator.

In [ ]:
# Refining the Lagrnagian mesh for a fix Eulerian grid.
# How does it affect the condition number of EBNH?
for i in range(1, 4 + 1):
    print('Resolution ratio: ', i)
    ds = dx / i
    N = int(round(2 * numpy.pi * radius / ds))
    theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1]
    x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
    body = pyibm.Body(x, y, grid=gridc)
    print(body)
    Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                              kernel=delta_kernel,
                              kernel_size=delta_kernel_size)
    EHat = Op @ R @ MHat
    HHat = csr_matrix(Op.T)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=1, L=LHat, M=MHat)
    EBNH = EHat @ BN @ HHat
    print_matrix_info(EBNH)
    print(condition_number(EBNH))
    # plot_matrix(EBNH, cmap='viridis');

The condition number of the operator `EBNH` explodes as we refine only the Lagrangian mesh while keeping the same back grid spacing.

In [ ]:
# Rotating the Lagrangian markers.
# How does it affect the condition number of EBNH?
num = 4
for i in range(num):
    print('Rotating case ', i)
    ds = dx
    N = int(round(2 * numpy.pi * radius / ds))
    epsilon = i * numpy.radians(45.0) / num
    theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1] + epsilon
    x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
    body = pyibm.Body(x, y, grid=gridc)
    Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                              kernel=delta_kernel,
                              kernel_size=delta_kernel_size)
    EHat = Op @ R @ MHat
    HHat = csr_matrix(Op.T)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=3, L=LHat, M=MHat)
    EBNH = EHat @ BN @ HHat
    print_matrix_info(EBNH)
    print(condition_number(EBNH))
    # plot_matrix(EBNH, cmap='viridis');

The condition number of the operator `EBNH` is not really influenced by the location of the markers on the surface.